In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from itertools import combinations

import time


In [26]:
EDGE_DRIVER_PATH = "edgedriver_win32/msedgedriver.exe"
driver = webdriver.Edge(service=Service(EDGE_DRIVER_PATH))

In [27]:
wait = WebDriverWait(driver, 10)

In [49]:
driver.get("https://rustypot.com/coinflip")

In [297]:
def get_possible_bet_values(inventory_items_values, max_items=3):
    possible_bet_values = {}
    for r in range(1, max_items):
        for combo in combinations(inventory_items_values, r):
            s = sum(combo)
            max_tax = s * 2 * 0.05
            good_combo = True
            for item_value in combo:
                if item_value < max_tax:
                    good_combo = False
            if good_combo:
                possible_bet_values[s] = combo
    possible_bet_values_sorted = dict(sorted(possible_bet_values.items()))
    
    return possible_bet_values_sorted

In [299]:
def get_inventory_items():
    create_a_game_buttom_locator = (By.CLASS_NAME, "createCoinflipButton")
    create_a_game_buttom = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(create_a_game_buttom_locator)
    )
    create_a_game_buttom.click()
    inventory_items_div_locator = (By.CLASS_NAME, "depositInventory")
    inventory_items_div = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(inventory_items_div_locator)
    )
    time.sleep(1)
    inventory_items = inventory_items_div.find_elements(By.CLASS_NAME, "InventoryItem")
    inventory_items_values = []
    for inventory_item in inventory_items:
        inventory_items_values.append(float(inventory_item.text[1:]))
    
    driver.get("https://rustypot.com/coinflip")
    
    print("inventory items gathered: ", inventory_items_values)
    
    return inventory_items_values

In [298]:
inventory_items_values = [62.31, 31.75, 31.75, 20.86, 17.38, 13.34, 13.34]

# inventory_items = [i for i in range(1, 21, 3)]
possible_bet_values = get_possible_bet_values(inventory_items_values)
possible_bet_values

{13.34: (13.34,),
 17.38: (17.38,),
 20.86: (20.86,),
 26.68: (13.34, 13.34),
 30.72: (17.38, 13.34),
 31.75: (31.75,),
 34.2: (20.86, 13.34),
 38.239999999999995: (20.86, 17.38),
 45.09: (31.75, 13.34),
 49.129999999999995: (31.75, 17.38),
 52.61: (31.75, 20.86),
 62.31: (62.31,),
 63.5: (31.75, 31.75),
 75.65: (62.31, 13.34),
 79.69: (62.31, 17.38),
 83.17: (62.31, 20.86),
 94.06: (62.31, 31.75)}

In [302]:
try:
    inventory_items_values = get_inventory_items()
except Exception as e:
    driver.get("https://rustypot.com/coinflip")
    time.sleep(2)
    inventory_items_values = get_inventory_items()
finally:
    if len(inventory_items_values) == 0:
        get_inventory_items()
        
possible_bet_values = get_possible_bet_values(inventory_items_values)
print(possible_bet_values)

inventory items gathered:  [319.48, 226.49, 111.79, 88.89, 87.05, 80.24, 63.2, 62.37, 62.37, 61.25, 58.23, 50.98, 46.94, 44.07, 43.33, 40.25, 38.73, 38.44, 37.81, 31.75, 31.75, 28.05, 23.11, 21.7, 21.13, 20.77, 20.32, 19.04, 18.9, 18.76, 18.24, 17.38, 17.0, 14.95, 14.22, 12.78, 12.78, 12.03, 12.0, 9.63, 9.28, 8.95, 8.21, 8.17, 5.06, 4.93, 4.23, 4.08, 3.5, 3.21, 1.31, 1.28]
{1.28: (1.28,), 1.31: (1.31,), 2.59: (1.31, 1.28), 3.21: (3.21,), 3.5: (3.5,), 4.08: (4.08,), 4.23: (4.23,), 4.49: (3.21, 1.28), 4.52: (3.21, 1.31), 4.78: (3.5, 1.28), 4.8100000000000005: (3.5, 1.31), 4.93: (4.93,), 5.06: (5.06,), 5.36: (4.08, 1.28), 5.390000000000001: (4.08, 1.31), 5.510000000000001: (4.23, 1.28), 5.540000000000001: (4.23, 1.31), 6.21: (4.93, 1.28), 6.24: (4.93, 1.31), 6.34: (5.06, 1.28), 6.369999999999999: (5.06, 1.31), 6.71: (3.5, 3.21), 7.29: (4.08, 3.21), 7.44: (4.23, 3.21), 7.58: (4.08, 3.5), 7.73: (4.23, 3.5), 8.14: (4.93, 3.21), 8.17: (8.17,), 8.21: (8.21,), 8.27: (5.06, 3.21), 8.31: (4.23, 4

In [306]:
bets = []
counter = 0
while True:

    active_coinflips = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ActiveCoinflips")))
    coinflip_row_divs = active_coinflips.find_elements(By.CLASS_NAME, "coinflip")
    
    # print(f"{len(coinflip_divs)} active games")
    for coinflip_div in coinflip_row_divs:
        coinflip_value = float(coinflip_div.get_attribute("coinflip-value"))
        coinflip_items = coinflip_div.find_element(By.CLASS_NAME, "coinflipGameItems")
        
        if "items" in coinflip_items.text:
            # print("more than 5 items...")
            continue
        img_elements = coinflip_items.find_elements(By.TAG_NAME, "img")
        
        coinflip_items_values = []
        for img in img_elements:
            title = img.get_attribute("data-original-title")
            if "|" in title:
                price_str = title.split("$")[-1]
                price = float(price_str)
                coinflip_items_values.append(price)
    
        possible_tax_value = coinflip_value * 2 * 0.05
        cheapest_item_value = coinflip_items_values[-1]
        
        if cheapest_item_value < possible_tax_value:
            # print(f"not profitable")
            continue
        
        # to join a cf, i need to deposit minimum value
        for bet_value in possible_bet_values:
            if  coinflip_value * 0.89 < bet_value < coinflip_value * 0.92 and coinflip_value not in bets:
                bets.append(coinflip_value)

                print(f"{coinflip_value=} -> "
                      f"{coinflip_items_values=} -> "
                      f"Bet items= {possible_bet_values[bet_value]}")
                break
        # print(f"{coinflip_value=} -> {possible_tax_value=:.2f} -> {cheapest_item_value=:.2f}")
    counter += 1
    # if counter % 100 == 0:
    #     print("active - ", counter)
    time.sleep(1)

coinflip_value=2.91 -> coinflip_items_values=[2.91] -> Bet items= (1.31, 1.28)
coinflip_value=34.35 -> coinflip_items_values=[14.57, 12.0, 7.78] -> Bet items= (21.7, 8.95)
coinflip_value=40.53 -> coinflip_items_values=[21.13, 12.78, 6.62] -> Bet items= (21.13, 14.95)
coinflip_value=58.23 -> coinflip_items_values=[58.23] -> Bet items= (37.81, 14.22)
coinflip_value=3.81 -> coinflip_items_values=[1.79, 0.69, 0.69, 0.64] -> Bet items= (3.5,)
coinflip_value=8.18 -> coinflip_items_values=[4.23, 3.95] -> Bet items= (4.08, 3.21)
coinflip_value=46.94 -> coinflip_items_values=[46.94] -> Bet items= (23.11, 18.76)
coinflip_value=31.75 -> coinflip_items_values=[31.75] -> Bet items= (19.04, 9.28)
coinflip_value=8.91 -> coinflip_items_values=[6.91, 2.0] -> Bet items= (4.93, 3.21)
coinflip_value=23.25 -> coinflip_items_values=[12.11, 11.14] -> Bet items= (20.77,)
coinflip_value=34.63 -> coinflip_items_values=[20.77, 13.86] -> Bet items= (18.9, 12.0)
coinflip_value=1.44 -> coinflip_items_values=[0.71, 

KeyboardInterrupt: 